# Пример теста

In [1]:
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize

In [2]:
# Исходные данные
p1 = 0.04   # контроль
p2 = 0.045  # тест, хотим поймать +0.5 п.п.
alpha = 0.05
power = 0.8

effect_size = proportion_effectsize(p1, p2)
analysis = NormalIndPower()
n = analysis.solve_power(effect_size, power=power, alpha=alpha, ratio=1, alternative='two-sided')

print(f"Нужно по {int(n)} наблюдений в каждую группу")

Нужно по 25530 наблюдений в каждую группу


In [3]:
from statsmodels.stats.proportion import proportions_ztest

In [4]:
clicks = [1149, 1278]
# clicks = [600, 675]
views = [25530, 25530]

zstat, pval = proportions_ztest(clicks, views)
print(f"p-value = {pval:.4f}")

p-value = 0.0073


 Интерпретация для кликов в  `clicks = [600, 675] (2.35% и 2.64%)` :
 
* p = 0.0334 < 0.05 → Разница статистически значима.

* CTR новой рассылки достоверно выше, чем у старой.

* НО! прирост составил только +0.29 п.п., а ты изначально рассчитывал тест на MDE = 0.5 п.п.

Мы рассчитали MDE = 0.5%, определили нужную выборку (~25k), провели тест и получили прирост CTR на 0.29 п.п.
Хотя прирост оказался меньше ожидаемого, результат статистически значим (p = 0.033), и мы дополнительно оценивали бизнес-ценность выката — стоимость рассылки, доход на клик и прочее


Что это означает для `clicks = [1149, 1278] (4.5% и 5%)` :

* Новый вариант достоверно повышает CTR на полпроцента, и это именно то, на что был рассчитан твой тест изначально.

* Разница статистически значима, т.к. p-value < 0.05.

* Это говорит о том, что с вероятностью > 99% такая разница не случайна.

В A/B‑тесте CTR вырос с 4.5% до 5.0%, что дало разницу в +0.5 п.п. При этом p-value = 0.0073, что говорит о статистически значимом эффекте. Так как мы изначально рассчитывали тест на MDE = 0.5 п.п. и достигли этого значения, мы приняли решение выкатить обновлённую рассылку на всех пользователей

* если p-value < 0.05 и наблюдаемый эффект ≥ MDE → результат статистически значим

* если эффект < MDE → даже если p-value низкое, это малозначимый прирост

Я всегда связываю MDE с ошибками I и II рода. Перед запуском эксперимента рассчитываю MDE и нужный объём выборки при α = 0.05 и Power = 0.8. Это помогает избежать ложных выводов. MDE — не просто технический параметр, а выражение минимального бизнес-значимого эффекта, который мы хотим зафиксировать

| Тип ошибки             | Что это?                                        | Где участвует?                                                |
| ---------------------- | ----------------------------------------------- | ------------------------------------------------------------- |
| **Ошибка I рода (α)**  | Ложно отклонили H₀ (нашли эффект, которого нет) | Определяет **уровень значимости**, чаще всего **5%**          |
| **Ошибка II рода (β)** | Не отклонили H₀, хотя эффект есть               | Определяет **мощность теста**: Power = 1 - β (обычно **80%**) |


# Пример - 2

Оценить минимальный размер выборки для A/B-теста, необходимый для обнаружения заданных эффектов в метрике `revenue` с учётом:

* Ошибки первого рода (α = 0.05)

* Ошибки второго рода (β = 0.2, т.е. мощность теста 80%).

In [5]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [6]:
def estimate_sample_size(df, metric_name, effects, alpha=0.05, beta=0.2):
    """
    Оцениваем sample size для списка эффектов.
    
    Параметры:
    df - pd.DataFrame, датафрёйм с данными
    metric_name - str, название столбца с целевой метрикой
    effects - List[float], список ожидаемых эффектов (например, [1.03] - увеличение на 3%)
    alpha - float, ошибка первого рода (значимость)
    beta - float, ошибка второго рода (1 - мощность)
    
    Возвращает:
    pd.DataFrame со столбцами ['effect', 'sample_size']
    """
    # Рассчитываем параметры распределения метрики
    mu = df[metric_name].mean()
    sigma = df[metric_name].std(ddof=0)  # стандартное отклонение генеральной совокупности
    
    # Критические значения для Z-статистики
    z_alpha = norm.ppf(1 - alpha/2)  # для двустороннего теста
    z_beta = norm.ppf(1 - beta)
    
    sample_sizes = []
    for effect in effects:
        # Рассчитываем абсолютный размер эффекта
        absolute_effect = mu * (effect - 1)
        
        # Формула для расчета размера выборки на одну группу
        n = (2 * sigma**2 * (z_alpha + z_beta)**2) / (absolute_effect**2)
        
        # Округляем до ближайшего целого в большую сторону
        n = int(np.ceil(n))
        sample_sizes.append(n)
    
    return pd.DataFrame({'effect': effects, 'sample_size': sample_sizes})

In [7]:
np.random.seed(42)
data = np.random.normal(loc=100, scale=15, size=1000)  # нормальное распределение
df = pd.DataFrame({'revenue': data})

# Параметры для теста
effects = [1.01, 1.03, 1.05, 1.10]  # эффекты: +1%, +3%, +5%, +10%
alpha = 0.05
beta = 0.2

# Вызываем функцию
result = estimate_sample_size(df, 'revenue', effects, alpha, beta)
print(result)

   effect  sample_size
0    1.01         3364
1    1.03          374
2    1.05          135
3    1.10           34


Почему чем больше эффект, тем меньше нужно данных?
1. <b>Большой эффект</b> (например, +10%) легче обнаружить статистически — разница между группами будет очевидной даже на малой выборке.

2. Маленький эффект (например, +1%) требует огромной выборки, потому что:

    * Случайные колебания могут "замаскировать" слабый эффект,
    * Нужно больше данных, чтобы доказать, что разница не случайна.


<b>Что делать, если выборка слишком большая?</b>

1. Увеличить допустимую ошибку (например, снизить мощность до 70% или α до 10%).
2. Укрупнить метрику (например, считать не конверсию, а средний чек).
3. Усилить эффект (например, тестировать радикальные изменения, а не мелкие правки).



<b>Ключевые выводы:</b>

* Чем меньше эффект, тем больше выборка нужна для его обнаружения (например, для +1% требуется 3364 наблюдения на группу).

* Крупные эффекты (например, +10%) обнаруживаются при малых выборках (34 наблюдения).

* Общий размер эксперимента: удвоенное значение (например, для +3% нужно 374 * 2 = 748 наблюдений).